In [1]:
from time import time
import lines_dataset as lds

In [2]:
data = lds.Dataset.read('random1')
t0 = time()
for i, x in data.samples().enumerate():
  t1 = time()
  t_mean = (t1 - t0) / (i + 1)
  print(f'\r{i} [{1e3*t_mean:.2f}ms per sample]', end='', flush=True)

5453 [1.87ms per sample]

KeyboardInterrupt: 